In [3]:
!pip install opencv-python

In [6]:
!pip install numpy==1.24.3 pandas==2.0.3

  Obtaining dependency information for numpy==1.24.3 from https://files.pythonhosted.org/packages/ee/6c/7217a8844dfe22e349bccbecd35571fa72c5d7fe8b33d8c5540e8cc2535c/numpy-1.24.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for pandas==2.0.3 from https://files.pythonhosted.org/packages/8f/bb/aea1fbeed5b474cb8634364718abe9030d7cc7a30bf51f40bd494bbc89a2/pandas-2.0.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for tzdata>=2022.1 from https://files.pythonhosted.org/packages/5c/23/c7abc0ca0a1526a0774eca151daeb8de62ec457e77262b66b359c3c7679e/tzdata-2025.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 583.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 541.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 829.0 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
   

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os
import logging


In [10]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [11]:
# Define constants
IMG_SIZE = 48
BATCH_SIZE = 32
EPOCHS = 40
NUM_CLASSES = 7
CLASSES = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
DATA_DIR = 'fer2013'
MODEL_PATH = 'facial_expression_model.h5'


In [12]:
# Check for dataset
if not os.path.exists(DATA_DIR):
    logger.error(f"Dataset directory '{DATA_DIR}' not found. Please download FER2013 from Kaggle.")
    raise FileNotFoundError(f"Directory '{DATA_DIR}' not found.")

In [13]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
# Load training and validation data
try:
    train_generator = train_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'train'),
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=CLASSES
    )
    validation_generator = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'test'),
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=CLASSES
    )
except Exception as e:
    logger.error(f"Error loading dataset: {e}")
    raise

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [15]:
# Build CNN model
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
# Train model
try:
    logger.info("Building and training model...")
    model = build_model()
    model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        verbose=1
    )
    model.save(MODEL_PATH)
    logger.info(f"Model saved to {MODEL_PATH}")
except Exception as e:
    logger.error(f"Training failed: {e}")
    raise

2025-07-18 21:41:36,356 - INFO - Building and training model...


Epoch 1/40
898/898 ━━━━━━━━━━━━━━━━━━━━ 55s 59ms/step - accuracy: 0.2558 - loss: 2.0796 - val_accuracy: 0.3323 - val_loss: 1.6702
Epoch 2/40
898/898 ━━━━━━━━━━━━━━━━━━━━ 54s 60ms/step - accuracy: 0.3461 - loss: 1.6460 - val_accuracy: 0.4039 - val_loss: 1.5016
Epoch 3/40
898/898 ━━━━━━━━━━━━━━━━━━━━ 53s 60ms/step - accuracy: 0.3991 - loss: 1.5381 - val_accuracy: 0.4374 - val_loss: 1.4085
Epoch 4/40
898/898 ━━━━━━━━━━━━━━━━━━━━ 54s 60ms/step - accuracy: 0.4202 - loss: 1.4787 - val_accuracy: 0.4482 - val_loss: 1.4298
Epoch 5/40
898/898 ━━━━━━━━━━━━━━━━━━━━ 57s 64ms/step - accuracy: 0.4433 - loss: 1.4394 - val_accuracy: 0.4646 - val_loss: 1.3711
Epoch 6/40
898/898 ━━━━━━━━━━━━━━━━━━━━ 54s 60ms/step - accuracy: 0.4625 - loss: 1.4032 - val_accuracy: 0.4802 - val_loss: 1.3602
Epoch 7/40
898/898 ━━━━━━━━━━━━━━━━━━━━ 54s 60ms/step - accuracy: 0.4740 - loss: 1.3809 - val_accuracy: 0.5274 - val_loss: 1.2337
Epoch 8/40
898/898 ━━━━━━━━━━━━━━━━━━━━ 55s 61ms/step - accuracy: 0.4801 - loss: 1.3529 - 

2025-07-18 22:19:18,322 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
2025-07-18 22:19:18,393 - INFO - Model saved to facial_expression_model.h5


In [ ]:
# Function to predict expression from an image
def predict_expression(image_path, model, cascade_path='haarcascade_frontalface_default.xml'):
    try:
        # Load Haar cascade for face detection
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + cascade_path)
        if face_cascade.empty():
            logger.error("Failed to load Haar cascade file.")
            raise FileNotFoundError("Haar cascade file not found.")

        # Load and preprocess image
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            logger.error(f"Failed to load image: {image_path}")
            raise ValueError("Invalid image path.")
        
        faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)
        if len(faces) == 0:
            logger.warning("No faces detected in the image.")
            return "No face detected"

        # Process first detected face
        (x, y, w, h) = faces[0]
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        face = face.astype('float32') / 255.0
        face = np.expand_dims(face, axis=(0, -1))  # Shape: (1, 48, 48, 1)

        # Predict
        prediction = model.predict(face)
        expression = CLASSES[np.argmax(prediction)]
        logger.info(f"Predicted expression: {expression}")
        return expression
    except Exception as e:
        logger.error(f"Error predicting expression: {e}")
        return None

In [ ]:
# Function for real-time webcam prediction
def webcam_predict(model, cascade_path='haarcascade_frontalface_default.xml'):
    try:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + cascade_path)
        if face_cascade.empty():
            logger.error("Failed to load Haar cascade file.")
            raise FileNotFoundError("Haar cascade file not found.")

        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            logger.error("Failed to open webcam.")
            raise RuntimeError("Webcam not accessible.")

        while True:
            ret, frame = cap.read()
            if not ret:
                logger.warning("Failed to capture frame.")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            for (x, y, w, h) in faces:
                face = gray[y:y+h, x:x+w]
                face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
                face = face.astype('float32') / 255.0
                face = np.expand_dims(face, axis=(0, -1))
                
                prediction = model.predict(face)
                expression = CLASSES[np.argmax(prediction)]
                
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, expression, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            cv2.imshow('Facial Expression Recognition', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    except Exception as e:
        logger.error(f"Error in webcam prediction: {e}")

In [ ]:
if __name__ == "__main__":
    # Load trained model for testing
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        logger.info("Model loaded successfully!")
        
        # Example: Predict expression from a single image
        # Replace 'path_to_image.jpg' with an actual image path
        # test_image = 'fer2013/test/happy/0001.jpg'
        # result = predict_expression(test_image, model)
        # print(f"Predicted expression: {result}")

        # Real-time webcam prediction
        logger.info("Starting webcam prediction (press 'q' to quit)...")
        webcam_predict(model)
    except Exception as e:
        logger.error(f"Error in main: {e}")